In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
# from keras.optimizers import Adam,RMSprop,SGD,Adamax,Adagrad,Adadelta,Nadam

In [13]:
# Load the data and preprocess it
df = pd.read_csv('data.csv')

In [14]:
df

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


In [15]:
# Preprocess the 'Blood Pressure' column
df[['Systolic', 'Diastolic']] = df['Blood Pressure'].str.split('/', expand=True)
df[['Systolic', 'Diastolic']] = df[['Systolic', 'Diastolic']].astype(float)

In [16]:
# Drop the original 'Blood Pressure' column and 'Gender' column (as it is not needed for the neural network)
df.drop(['Person ID', 'Occupation','Quality of Sleep','Physical Activity Level','BMI Category','Sleep Disorder','Blood Pressure', 'Gender'], axis=1, inplace=True)

In [17]:
df

,Age,Sleep Duration,Stress Level,Heart Rate,Daily Steps,Systolic,Diastolic
0,27,6.1,6,77,4200,126.0,83.0
1,28,6.2,8,75,10000,125.0,80.0
2,28,6.2,8,75,10000,125.0,80.0
3,28,5.9,8,85,3000,140.0,90.0
4,28,5.9,8,85,3000,140.0,90.0
...,...,...,...,...,...,...,...
369,59,8.1,3,68,7000,140.0,95.0
370,59,8.0,3,68,7000,140.0,95.0
371,59,8.1,3,68,7000,140.0,95.0
372,59,8.1,3,68,7000,140.0,95.0


In [20]:
# Normalize the health vitals data (optional)
scaler = MinMaxScaler()
columns_to_normalize = ['Heart Rate', 'Stress Level', 'Systolic', 'Diastolic', 'Sleep Duration']
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [21]:
# Define the weights
weights = {
    'Heart Rate': 0.3,
    'Stress Level': 0.2,
    'Systolic': 0.2,
    'Diastolic': 0.2,
    'Sleep Duration': 0.3
}

# Calculate the fitness score using the defined weights
df['Fitness Score'] = df[columns_to_normalize].dot(pd.Series(weights))

In [22]:
# Save the updated dataset with the fitness score
df.to_csv('health_vitals_data.csv', index=False)

In [23]:
df1=pd.read_csv('health_vitals_data.csv')

In [24]:
df1

,Age,Sleep Duration,Stress Level,Heart Rate,Daily Steps,Systolic,Diastolic,Fitness Score
0,0.00000,0.111111,0.6,0.571429,0.171429,0.407407,0.40,0.486243
1,0.03125,0.148148,1.0,0.476190,1.000000,0.370370,0.25,0.511376
2,0.03125,0.148148,1.0,0.476190,1.000000,0.370370,0.25,0.511376
3,0.03125,0.037037,1.0,0.952381,0.000000,0.925926,0.75,0.832011
4,0.03125,0.037037,1.0,0.952381,0.000000,0.925926,0.75,0.832011
...,...,...,...,...,...,...,...,...
369,1.00000,0.851852,0.0,0.142857,0.571429,0.925926,1.00,0.683598
370,1.00000,0.814815,0.0,0.142857,0.571429,0.925926,1.00,0.672487
371,1.00000,0.851852,0.0,0.142857,0.571429,0.925926,1.00,0.683598
372,1.00000,0.851852,0.0,0.142857,0.571429,0.925926,1.00,0.683598


In [25]:
# Split the data into features (X) and target (y)
X = df.drop(columns=['Fitness Score']).values
y = df['Fitness Score'].values

In [47]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

In [43]:
# Build the Neural Network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer for regression with linear activation

In [68]:
# Compile the model
model.compile(optimizer='Nadam', loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error'])

In [77]:
# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/200
8/8 [==============================] - 0s 13ms/step - loss: 6.3037e-07 - mean_squared_error: 6.3037e-07 - mean_absolute_error: 6.1563e-04 - val_loss: 1.7116e-05 - val_mean_squared_error: 1.7116e-05 - val_mean_absolute_error: 0.0017
Epoch 2/200
8/8 [==============================] - 0s 7ms/step - loss: 1.0897e-06 - mean_squared_error: 1.0897e-06 - mean_absolute_error: 8.9393e-04 - val_loss: 1.9427e-05 - val_mean_squared_error: 1.9427e-05 - val_mean_absolute_error: 0.0019
Epoch 3/200
8/8 [==============================] - 0s 7ms/step - loss: 1.2303e-06 - mean_squared_error: 1.2303e-06 - mean_absolute_error: 9.4789e-04 - val_loss: 1.4613e-05 - val_mean_squared_error: 1.4613e-05 - val_mean_absolute_error: 0.0023
Epoch 4/200
8/8 [==============================] - 0s 7ms/step - loss: 3.9339e-06 - mean_squared_error: 3.9339e-06 - mean_absolute_error: 0.0018 - val_loss: 1.8791e-05 - val_mean_squared_error: 1.8791e-05 - val_mean_absolute_error: 0.0032
Epoch 5/200
8/8 [==============

In [78]:
# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)


print(f'Mean Squared Error: {mse}')
print(f'R-squared (R2) Score: {r2}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

3/3 [==============================] - 0s 3ms/step
Mean Squared Error: 1.0493614447657894e-05
R-squared (R2) Score: 0.9994930896059913
Mean Absolute Error (MAE): 0.0017516868583741137
Root Mean Squared Error (RMSE): 0.0032393848872367564
